# 2. Create the total map for all frequencies by adding CMB realization, Synchrotron realization, and Noise realization together

In [2]:
# !pip install skyclean --upgrade
import os
import healpy as hp
import skyclean as sc
# from skyclean import hp_alm_2_mw_alm, arcmin_to_radians, reduce_hp_map_resolution

/home/matthew/ENTER/envs/skyclean‐fix/lib/python3.13/site-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.6.2 is installed, but it is not compatible with the installed jaxlib version 0.5.3, so it will not be used.
  warnings.warn(


In [3]:
# Skyclean functions to create total map

import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
# import os
from astropy.io import fits #For beam deconvolution

def reduce_hp_map_resolution(hp_map, lmax, nside):
    """
    Processes a Healpix map by converting it to spherical harmonics and back,
    and reducing the resolution.
    
    Parameters:
        map_data (numpy.ndarray): Input map data.
        lmax (int): Maximum multipole moment for spherical harmonics.
        nside (int): Desired nside resolution for the output map.
        
    Returns:
        numpy.ndarray: Processed map data.
    """
    hp_alm = hp.map2alm(hp_map, lmax=lmax)
    processed_map = hp.alm2map(hp_alm, nside=nside)
    return processed_map, hp_alm



# slightly adapted from standard skyclean beam_deconvolution function to allow convolution only
def beam_convolution(hp_map, frequency, lmax, standard_fwhm_rad, 
                     beam_path_template = "HFI_beams/"+ f"Bl_T_R3.01_fullsky_{{frequency}}x{{frequency}}.fits", 
                     LFI_beam_fwhm = {"030": 32.33, "044": 27.01, "070": 13.25}, deconv = True):
    """
    Performs beam deconvolution on the given CMB map data and returns the deconvolved map.

    Parameters:
        cmb_map (fits): CMB map data.
        frequency (str): Frequency identifier (e.g., "030", "044").
        lmax (int): Maximum multipole moment.
        standard_fwhm_rad (float): Standard beam full-width half-maximum in radians.
        beam_path (str): Path to the beam data file specific to the frequency.
        LFI_beam_fwhm (dict): Dictionary of beam full-width half-maximum (FWHM) in arcminutes for LFI frequencies.
    Returns:
      deconvolved_map (fits): The deconvolved CMB map.
    """

    nside = hp.get_nside(hp_map)
    alm = hp.map2alm(hp_map, lmax=lmax)
    # Standard beam for the desired FWHM
    Standard_bl = hp.sphtfunc.gauss_beam(standard_fwhm_rad, lmax=lmax-1, pol=False)
    # Pixel window function
    pixwin = hp.sphtfunc.pixwin(nside, lmax=lmax, pol=False)
    beam_path = beam_path_template.format(frequency = frequency)
    # if CMB, deconvolve
    if deconv: 
        # LFI beam deconvolution
        if frequency in {"030", "044", "070"}:
            # Deconvolution for lower frequencies
            fwhm_rad = np.radians(LFI_beam_fwhm[frequency] / 60)
            bl = hp.sphtfunc.gauss_beam(fwhm_rad, lmax=lmax-1, pol=False)
            alm_deconv = hp.almxfl(alm, 1/bl)
        # HFI beam deconvolution
        else:
            # Deconvolution using FITS file for higher frequencies
            hfi = fits.open(beam_path)
            beam = hfi[1].data["TEMPERATURE"]
            alm_deconv = hp.almxfl(alm, 1/beam) # divide out the beam
    else:
        alm_deconv = alm
    # Divide out pixel window function
    alm_deconv = hp.almxfl(alm_deconv, 1/pixwin)
    # Apply standard beam
    alm_reconv = hp.almxfl(alm_deconv, Standard_bl)
    # Convert back to map
    hp_map_reconv = hp.alm2map(alm_reconv, nside=nside)
    
    return hp_map_reconv

In [4]:
# This function is not included in the Skyclean package because the storage directory and beam deconvolution process
#  are dependent on the user.
def create_and_save_total_map(frequency, realization, desired_lmax, directory="CMB_realizations"):
    """
    Processes the CMB, Synchrotron, and Noise maps for each frequency and realization, then combines them.

    Beam deconvolution is applied to the CMB map.
    The CMB, Synchrotron, and Noise maps are then reduced to the desired lmax.
    The reduced maps are then combined to create the total map.

    Parameters:
        frequency (str): frequency identifiers.
        realization (int): realization identifiers.
        desired_lmax (int): Maximum multipole moment for spherical harmonics.
        directory (str): Directory containing the input map files.
    """
    # Ensure the directory exists
    if not os.path.exists("data/CMB_total"):
        os.makedirs("data/CMB_total")

    print(f"Processing maps for frequency {frequency} and realization {realization}")
        
    # Define file paths
    CMB_file_path = f"{directory}/febecop_ffp10_lensed_scl_cmb_{frequency}_mc_{realization:04d}.fits"
    synchrotron_file_path = f"{directory}/COM_SimMap_synchrotron-ffp10-skyinbands-{frequency}_2048_R3.00_full.fits"
    noise_file_path = f"{directory}/ffp10_noise_{frequency}_full_map_mc_{realization:05d}.fits"
    
    # Read maps
    original_hp_CMB_map, cmb_header = hp.read_map(CMB_file_path, h = True)
    synchrotron, synchrotron_header = hp.read_map(synchrotron_file_path, h = True)
    noise, noise_header = hp.read_map(noise_file_path, h = True)

    # Remember to check the units of the maps by print(header) (CMB_K, MJy/sr, etc.)
    # The unit coversion: https://wiki.cosmos.esa.int/planckpla2015/index.php/UC_CC_Tables 
    #print(cmb_header)
    #print(synchrotron_header)
    #print(noise_header)
    
    if frequency == "545":
        unit_conversion = 58.0356
        original_hp_CMB_map = original_hp_CMB_map / unit_conversion
        synchrotron = synchrotron / unit_conversion
        noise =  noise / unit_conversion
    if frequency == "857":
        unit_conversion = 2.2681
        original_hp_CMB_map = original_hp_CMB_map / unit_conversion
        synchrotron = synchrotron / unit_conversion
        noise =  noise / unit_conversion
    #print(f"original @ {frequency}:{original_hp_CMB_map}")
    # Define your own beam function path
    HFI_beam_path = "HFI_beams/"+ f"Bl_T_R3.01_fullsky_{frequency}x{frequency}.fits"
    
    beam_decon_cmb = beam_convolution(original_hp_CMB_map, frequency, desired_lmax, sc.arcmin_to_radians(5)) # same shape
    beam_decon_synchrotron = beam_convolution(synchrotron, frequency, desired_lmax, sc.arcmin_to_radians(5), deconv = False) # sync already deconved

    
    #print(f"deconved @ {frequency}:{beam_decon_cmb}")
    
    # # Calculate nside based on lmax
    nside = desired_lmax // 2
            
    old_cmb,_ = sc.reduce_hp_map_resolution(original_hp_CMB_map, desired_lmax, nside)
    new_cmb,_ = sc.reduce_hp_map_resolution(beam_decon_cmb, desired_lmax, nside)
    new_synchrotron,_  = sc.reduce_hp_map_resolution(beam_decon_synchrotron, desired_lmax, nside)
    new_noise,_  = sc.reduce_hp_map_resolution(noise, desired_lmax, nside)
    #total map
    csn = new_cmb + new_synchrotron + new_noise 
    # Save processed maps
    map_dict = {
        "conved_CMB": old_cmb,
        "CMB": new_cmb,
        "Synchrotron": new_synchrotron,
        "Noise": new_noise,
        "CSN": csn
    }
    for map_type, _map in map_dict.items():
        filename = f"data/CMB_total/{map_type}_HP_Map_F{frequency}_L{desired_lmax}_R{realization:04d}.fits"
        hp.write_map(filename, _map, dtype="float64", overwrite=True)


In [5]:
# Usage of the create_and_save_total_map function

# Define frequencies and realizations
frequencies = ["030", "100", "353"]
#frequencies = ["030", "044", "070", "100", "143", "217", "353", "545", "857"]
realizations = list(range(1)) 

desired_lmax = 512
for frequency in frequencies:
    for realization in realizations:
        path = f"CMB_total/CSN_HP_Map_F{frequency}_L{desired_lmax}_R{realization:04d}.fits"
        if os.path.exists(path):
            print(f"File {path} already exists. Skipping download.")
            continue
        create_and_save_total_map(frequency, realization, desired_lmax, directory="data/CMB_realizations")



Processing maps for frequency 030 and realization 0


ValueError: cannot reshape array of size 1589952 into shape (1552,1024)

In [25]:
import pyssht
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np

In [48]:
import healpy as hp, pyssht, matplotlib.pyplot as plt

lmax  = 1024
L     = lmax       
nside = lmax // 2

cmb_map   = hp.read_map("data/CMB_total/CMB_HP_Map_F030_L1024_R0000.fits")
alm_hp    = hp.map2alm(cmb_map, lmax=lmax)     

mw_samples = pyssht.inverse(alm_hp, L=L,
                            Spin=0, Method='MW',
                            Reality=True)             

alm_back = pyssht.forward(mw_samples, L=L,
                          Spin=0, Method='MW',
                          Reality=True)                
           
print(alm_hp.shape, alm_back.shape)
cmb_map_back = hp.alm2map(alm_back, nside=nside, lmax=lmax)
hp.mollview(cmb_map_back - cmb_map, title='Residual (MW round-trip)')
plt.show()


(525825,) (1048576,)


ValueError: Wrong alm size.

525312.5
